In [1]:
from ragcar import Ragcar
from ragcar.utils import PromptTemplate

/home/pcn_rns/anaconda3/envs/rc_test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
generator = Ragcar(tool="text_generation", src="sllm", model="upstage/SOLAR-10.7B-Instruct-v1.0", prompt_template=PromptTemplate("{input} 수도는?"), lora_model_dir="/home/pcn_rns/ella/git_frameworks/axolotl/output_solor/exp_13" , adapter="lora")

INFO:text_generation:Loading SLLM model
INFO:sllm:model_n: upstage/SOLAR-10.7B-Instruct-v1.0, lora_path: /home/pcn_rns/ella/git_frameworks/axolotl/output_solor/exp_13, deepspeed: None, adapter_type: False, formatting: False
INFO:ragcar.models.base:Using device cuda with dtype torch.float16
INFO:ragcar.models.base:Using device cuda with dtype torch.float16
INFO:ragcar.models.base:patching _expand_mask in llama models
INFO:ragcar.models.base:model_kwargs: {'torch_dtype': torch.float16, 'load_in_8bit': False, 'device_map': 'auto'}


EOS: 2 / </s>
BOS: 1 / <s>
PAD: 2 / </s>
UNK: 0 / <unk>


Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.52it/s]
INFO:ragcar.models.base:resizing token embeddings to 32000
INFO:ragcar.models.base:model config: cuda
INFO:ragcar.models.base:GPU memory usage after model load: 4.735GB (+0.045GB cache, +21.476GB misc)
INFO:ragcar.models.base:converting modules to torch.float16 for flash attention
INFO:text_generation:Loaded SLLM model: <ragcar.models.sllm.SLLMCompletion object at 0x7fe3b1703970>


In [3]:
generator(input="대한민국")

TypeError: _get_params() got an unexpected keyword argument 'repetition_penalty'